In [1]:
import sys
sys.path.insert(0, '/workspace/brainbias/src')


In [2]:
from os import environ

from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel 
import lightning.pytorch as pl

from model import BERT 
from pl_model import LitBert

from utils.EthicsDataset import EthicsDataset
from utils.loading_data import multiple_dataset_loading, DEFAULT_DATASETS
from utils.constants import Sampling

import wandb
from lightning.pytorch.loggers import WandbLogger

In [3]:
from transformers import AutoModelForMaskedLM, RobertaTokenizer, RobertaModel, BertForSequenceClassification 

# DeBERTa
# tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
# model = AutoModel.from_pretrained("microsoft/deberta-v3-base")

# BERT - uncased
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = AutoModelForMaskedLM.from_pretrained("bert-base-cased")

# BERT - cased
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModel.from_pretrained("bert-base-cased")

# RoBERTa
# tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
# model = RobertaModel.from_pretrained('roberta-large')


In [4]:
# !pip install gputil
import torch
import GPUtil

torch.cuda.empty_cache()

GPUtil.showUtilization()



| ID | GPU | MEM |
------------------
|  0 |  0% | 37% |


In [5]:
    
logger = WandbLogger()

context = {'artifactspath': '/workspace/brainbias/artifacts',
            'batch_size': 10,
            'batches_per_epoch': 20,
            'checkpoint': 'bert-base-cased',
            'datapath': '/workspace/brainbias/data',
            'loss_names': ['cross-entropy', 'mse'],
            'loss_weights': [1.0, 1.0],
            'lr': 0.0006538379548447884,
            'num_epochs': 4,
            'num_samples_test':  1500,
            'num_samples_train': 1000,
            'only_train_head': False,
            'regularization_coef': 0.1,
            'regularize_from_init': False,
            'sampling_method': Sampling.LAST, # <Sampling.LAST: 1>,
            'shuffle_test': False,
            'shuffle_train': True,
            'train_datasets': ['EthicsDataset', 'DS000212_LFB_Dataset']}

dataloaders, train_head_dims = multiple_dataset_loading(tokenizer, context)

lit_model = LitBert(BERT(model, head_dims=train_head_dims), context)

# logger = WandbLogger(save_dir=context['artifactspath'], project="AISC_BB")
logger.log_hyperparams(context)

# train the model
trainer = pl.Trainer(
    limit_train_batches=context['batches_per_epoch'],
    max_epochs=context['num_epochs'],
    accelerator='auto',
    devices='auto',
    strategy='auto',
    logger=logger,
    log_every_n_steps=1,
    default_root_dir=context['artifactspath'],
    enable_checkpointing=False  # Avoid saving full model into a disk (GBs)
)

print('Fine tuning BERT...')
trainer.fit(lit_model, dataloaders)

data = EthicsDataset(context, tokenizer, is_train=False)
test_loader = DataLoader( data, batch_size=context['batch_size'], shuffle=context['shuffle_test'])

print('Testing on ETHICS...')
trainer.test(lit_model, dataloaders=test_loader)
logger.save()
wandb.finish()

AssertionError: 

In [5]:
def wandb_sweep():

    wandb.init()
    logger = WandbLogger()

    context = {'artifactspath': '/workspace/brainbias/artifacts',
               'batch_size': wandb.config.batch_size, # 15
               # 'batches_per_epoch': 15
               'checkpoint': 'bert-base-cased',
               'datapath': '/workspace/brainbias/data',
               'loss_names': ['cross-entropy', 'mse'],
               'loss_weights': [1.0, 1.0],
               'lr': wandb.config.lr, # 0.0006538379548447884,
               # 'num_epochs': 4,
               'num_samples_test':  1500, # 3885,
               'num_samples_train': 1000, # 13911,
               'only_train_head': False,
               'regularization_coef': 0.1,
               'regularize_from_init': False,
               'sampling_method': Sampling.LAST, # <Sampling.LAST: 1>,
               'shuffle_test': False,
               'shuffle_train': True,
               'train_datasets': ['EthicsDataset', 'DS000212_LFB_Dataset']}

    dataloaders, train_head_dims = multiple_dataset_loading(tokenizer, context)

    model = AutoModel.from_pretrained("bert-base-cased")
    lit_model = LitBert(BERT(model, head_dims = train_head_dims), context)

    # logger = WandbLogger(save_dir=context['artifactspath'], project="AISC_BB")
    logger.log_hyperparams(context)

    # train the model
    trainer = pl.Trainer(
        limit_train_batches=wandb.config.batches_per_epoch,
        max_epochs=wandb.config.num_epochs,
        accelerator='auto',
        devices='auto',
        strategy='auto',
        logger=logger,
        log_every_n_steps=1,
        default_root_dir=context['artifactspath'],
        enable_checkpointing=True  # Avoid saving full model into a disk (GBs)
    )
    
    print('Fine tuning BERT...')
    trainer.fit(lit_model, dataloaders)

    data = EthicsDataset(context, tokenizer, is_train=False)
    test_loader = DataLoader( data, batch_size=context['batch_size'], shuffle=context['shuffle_test'])
    
    print('Testing on ETHICS...')
    trainer.test(lit_model, dataloaders=test_loader)
    logger.save()
    wandb.finish()

In [6]:
sweep_config = {'method': 'random'}
parameters_dict = {# 'model': {'value': model},
                   'batch_size': {'values': [12]},
                   'batches_per_epoch': {'values': [19]},
                   'lr': {'distribution': 'uniform',
                          'min': 1e-5,
                          'max': 5e-5},
                   'num_epochs': {'values': [250]},
                    }

sweep_config['parameters'] = parameters_dict

sweep_id = wandb.sweep(sweep_config, project="Seong-AISC_BB")

wandb.agent(sweep_id, function = wandb_sweep, count = 100)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 9jxs9dnv
Sweep URL: https://wandb.ai/asdfasdfasdfdsafsd/Seong-AISC_BB/sweeps/9jxs9dnv


wandb: Agent Starting Run: ikzewm7q with config:
wandb: 	batch_size: 12
wandb: 	batches_per_epoch: 19
wandb: 	lr: 3.984818777527335e-05
wandb: 	num_epochs: 250
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: seonghahcho (asdfasdfasdfdsafsd). Use `wandb login --relogin` to force relogin


/root/mambaforge/envs/brainbias/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
wandb: WARNING Config item 'batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr' was locked by 'sweep' (ignored update).
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Fine tuning BERT...


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: qp8dlvc9 with config:
wandb: 	batch_size: 12
wandb: 	batches_per_epoch: 19
wandb: 	lr: 4.9089517179372296e-05
wandb: 	num_epochs: 250


/root/mambaforge/envs/brainbias/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
wandb: WARNING Config item 'batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr' was locked by 'sweep' (ignored update).
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/root/mambaforge/envs/brainbias/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:71: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | BERT | 109 M 
----------

Fine tuning BERT...


/root/mambaforge/envs/brainbias/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]